In [1]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [2]:
filepath = Path("../sample_files/ellipsoid.stl")
lv = LV.from_file(filepath)
lv.plot(show_edges=True)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [3]:
from project_heart.utils.spatial_utils import *
from project_heart.utils.vector_utils import *

In [4]:
lv.identify_base_and_apex_surfaces()
lv.identify_epi_endo_surfaces(90)
lv.plot(scalars=LV_MESH_DATA.APEX_BASE_REGIONS.value, categorical=True)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [7]:
e1=15
b1 = 1
b2 = 89

endo_epi = np.copy(lv.get(GEO_DATA.SURF_POINT_DATA, LV_MESH_DATA.EPI_ENDO_GUESS))
lvsurf = lv.get_surface_mesh()
# Get edges
edges = lv.mesh.extract_feature_edges(e1)
edges = edges.extract_largest()
edges = edges.extract_largest()
edge_pts = edges.points
est_base = centroid(edge_pts)
est_radius = radius(edge_pts)
# select pts close to est_base based on dist threshold on axis
pts = lvsurf.points
lvnormal = lv.get_normal()
axis = np.where(lvnormal==np.max(lvnormal))[0]
d_base = np.abs(est_base[axis] - pts[:, axis])
ioi = np.where(d_base <= 0.1)[0]
# filter selected pts based on surface angle
surfnormals = lvsurf.compute_normals(inplace=True)
surf_normals = lvsurf.get_array("Normals", "points")
base_angles = lv.compute_angles_wrt_normal(surf_normals[ioi], check_orientation=False, degrees=False)
ioi = ioi[np.where(base_angles < np.radians(180))[0]]
# filter by endo
ioi = ioi[np.where(endo_epi[ioi] != LV_SURFS.ENDO)]
# identify final surfaces
endo_epi_base = np.copy(endo_epi)
endo_epi_base[ioi] = LV_SURFS.BASE
# map to 'global' mesh ids
id_map = lv.get_surface_id_map_from_mesh()
endo_epi_base_mesh = np.zeros(lv.mesh.n_points)
endo_epi_base_mesh[id_map] = endo_epi_base
# add data to mesh
lv._surface_mesh.point_data[LV_MESH_DATA.SURFS.value] = endo_epi_base
lv.mesh.point_data[LV_MESH_DATA.SURFS.value] = endo_epi_base_mesh
#plot
lv.plot("surface", 
        LV_MESH_DATA.SURFS.value,
        vnodes=[
          (LV_VIRTUAL_NODES.APEX, "green"),
          (LV_VIRTUAL_NODES.BASE, "red"),
          ],
        categorical=True)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [8]:
lv.mesh.save("./test.vtk")